In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from trl import SFTTrainer
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, TextStreamer
from unsloth import FastLanguageModel, is_bfloat16_supported

# Define model and training parameters
max_seq_length = 1024
dtype = None
load_in_4bit = True

# Load pre-trained model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Apply LoRA configurations (without merging)
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    use_gradient_checkpointing=True,
    random_state=3407,
)

# Define the prompt format for the task
career_prompt = """Given the individual's profile:
{input}

Recommend a suitable career path and explain why.

Response: {output}"""

# Function to format dataset prompts
def format_prompts(examples):
    formatted = []
    for input_data, output in zip(examples["Input"], examples["Output"]):
        text = career_prompt.format(
            input=input_data,
            output=output
        )
        formatted.append(text)
    return {"text": formatted}

# Load datasets and format prompts
train_dataset = load_dataset("csv", data_files="train dataset llama.csv", split="train")
train_dataset = train_dataset.map(format_prompts, batched=True)

test_dataset = load_dataset("csv", data_files="test dataset llama.csv", split="train")
test_dataset = test_dataset.map(format_prompts, batched=True)

# Training arguments with checkpointing every 10 steps
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_ratio=0.03,
    max_steps=100,
    learning_rate=1e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.05,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="outputs",
    report_to=[],
    save_steps=10,  # Save checkpoint every 10 steps
    save_total_limit=3  # Keep only the last 3 checkpoints
)

# Initialize Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

# Train the model
trainer_stats = trainer.train()

# Save the final model weights after training
#model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")  # Save model with adapter weights





🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

Map (num_proc=2):   0%|          | 0/1500 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 6,815,744


Step,Training Loss
1,2.125800
2,2.105900
3,2.080900
4,2.085600
5,2.053600
6,1.957500
7,1.859100
8,1.794800
9,1.705300
10,1.637400


In [ ]:
!pip uninstall fsspec huggingface_hub -y
!pip install fsspec huggingface_hub


Found existing installation: fsspec 2024.6.1
Uninstalling fsspec-2024.6.1:
  Successfully uninstalled fsspec-2024.6.1
Found existing installation: huggingface-hub 0.24.7
Uninstalling huggingface-hub-0.24.7:
  Successfully uninstalled huggingface-hub-0.24.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 19.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.0.2 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.10.0 which is incompatible.
torchaudio 2.5.0+cu121 requires torch==2.5.0, but you have torch 2.3.0 which is incompatible.
torchvision 0.20.0+cu121 requires torch==2.5.0, but you have torch 2.3.0 which is incompatible.


In [ ]:
model.push_to_hub_merged("kharshita590/llama-fine", tokenizer, save_method="merged_16bit")




Unsloth: You are pushing to hub, but you passed your HF username = kharshita590.
We shall truncate kharshita590/llama-fine to llama-fine
Unsloth: Will remove a cached repo with size 1.4K


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.29 out of 12.67 RAM for saving.


100%|██████████| 32/32 [05:45<00:00, 10.79s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving llama-fine/pytorch_model-00001-of-00004.bin...
Unsloth: Saving llama-fine/pytorch_model-00002-of-00004.bin...
Unsloth: Saving llama-fine/pytorch_model-00003-of-00004.bin...
Unsloth: Saving llama-fine/pytorch_model-00004-of-00004.bin...


README.md:   0%|          | 0.00/593 [00:00<?, ?B/s]

...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/kharshita590/llama-fine


In [ ]:

!pip install --upgrade peft


In [4]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
        

In [5]:
predictions = []
references = []

for example in test_dataset:
    input_text = example["text"]
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_seq_length).to(model.device)
    output = model.generate(**inputs, max_length=max_seq_length)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(generated_text)
    references.append(example["Output"])

# Calculate word-level accuracy
word_accuracies = []
for pred, ref in zip(predictions, references):
    pred_words = pred.split()
    ref_words = ref.split()
    correct_words = sum(1 for p, r in zip(pred_words, ref_words) if p == r)
    word_accuracy = correct_words / max(len(ref_words), 1)
    word_accuracies.append(word_accuracy)

average_word_accuracy = sum(word_accuracies) / len(word_accuracies)

# Calculate sentence-level accuracy
sentence_accuracies = [1 if pred == ref else 0 for pred, ref in zip(predictions, references)]
average_sentence_accuracy = sum(sentence_accuracies) / len(sentence_accuracies)

print(f"Word-Level Accuracy: {average_word_accuracy * 100:.2f}%")
print(f"Sentence-Level Accuracy: {average_sentence_accuracy * 100:.2f}%")

KeyboardInterrupt: 

In [ ]:
def generate_career_recommendation(profile):
    prompt = career_prompt.format(input=profile, output="")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        use_cache=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
test_profile = {'Problem_Solving_Approach': 'Logical, step-by-step', 'Comfort_with_Technical_Tasks': 'Uncomfortable', 'Grasp_of_Technical_Concepts': 'Takes time', 'Work_Process_Preference': 'Trial and error', 'Task_Management_Style': 'Excellent at multitasking', 'Stress_Handling': 'Calm, but slow', 'Team_Role': 'Collaborator', 'Interaction_Preference': 'Collaborating with others', 'Decision_Making_Style': 'Consulting others', 'Adaptability_to_Change': 'Adjusts easily', 'Interest_Activity': 'Designing creative projects', 'Preferred_Projects': 'Event planning', 'Work_Life_Balance': 'Structured 9-to-5', 'Career_Motivation': 'Creative freedom', 'Personality': 'Leadership'}

print(generate_career_recommendation(test_profile))

Given the individual's profile:
{'Problem_Solving_Approach': 'Logical, step-by-step', 'Comfort_with_Technical_Tasks': 'Uncomfortable', 'Grasp_of_Technical_Concepts': 'Takes time', 'Work_Process_Preference': 'Trial and error', 'Task_Management_Style': 'Excellent at multitasking', 'Stress_Handling': 'Calm, but slow', 'Team_Role': 'Collaborator', 'Interaction_Preference': 'Collaborating with others', 'Decision_Making_Style': 'Consulting others', 'Adaptability_to_Change': 'Adjusts easily', 'Interest_Activity': 'Designing creative projects', 'Preferred_Projects': 'Event planning', 'Work_Life_Balance': 'Structured 9-to-5', 'Career_Motivation': 'Creative freedom', 'Personality': 'Leadership'}

Recommend a suitable career path and explain why.

Response:  Given your personality trait of Leadership and your work preferences, you are well-suited for a career in Business and Finance. Your Logical, step-by-step approach to problem-solving and your Trial and error work process preference align with

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install peft
!pip install bitsandbytes
!pip install xformers
!pip install trl
!pip install unsloth


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-uyg1kfr1/unsloth_2842540c93d5495d8da8133039a088ea
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-uyg1kfr1/unsloth_2842540c93d5495d8da8133039a088ea
  Resolved https://github.com/unslothai/unsloth.git to commit 49ae6194122b594a7054da0bfd6f387cf720f40f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.2 MB/s eta 0:00:00
  

In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-1fywlfx4/unsloth_2d861864ba2a4da6967b74d9910bb35d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-1fywlfx4/unsloth_2d861864ba2a4da6967b74d9910bb35d
  Resolved https://github.com/unslothai/unsloth.git to commit 49ae6194122b594a7054da0bfd6f387cf720f40f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import login
access_token = HF_TOKEN
login(token=access_token)